### Identifier les valeurs aberrantes dans `products.csv`

Utiliser `pandas` pour charger et analyser les données du fichier CSV `products.csv`. Repérer les valeurs aberrantes (ordre de grandeur : quelques centaines).

In [85]:
from pathlib import Path

import numpy as np
import pandas as pd

In [34]:
DATA_DIR = Path("../../data")
product_file_path = DATA_DIR / "products.csv"

In [35]:
product_df = pd.read_csv(product_file_path, low_memory=False)

In [36]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320772 entries, 0 to 320771
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          320772 non-null  int64  
 1   code                320749 non-null  object 
 2   fat_100g            243891 non-null  float64
 3   saturated-fat_100g  229554 non-null  float64
 4   sugars_100g         244971 non-null  float64
 5   fiber_100g          200886 non-null  float64
 6   proteins_100g       259922 non-null  float64
 7   salt_100g           255510 non-null  float64
 8   sodium_100g         255463 non-null  float64
 9   autre               320772 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 24.5+ MB


In [9]:
product_df.describe()

,Unnamed: 0,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre
count,320772.000000,243891.000000,229554.000000,244971.000000,200886.000000,259922.000000,255510.000000,255463.000000,320772.000000
mean,160385.500000,12.730379,5.129932,16.003484,2.862111,7.075940,2.028624,0.798815,65.861511
std,92599.044612,17.578747,8.014238,22.327284,12.867578,8.409054,128.269454,50.504428,32.091021
min,0.000000,0.000000,0.000000,-17.860000,-6.700000,-800.000000,0.000000,0.000000,0.000000
25%,80192.750000,0.000000,0.000000,1.300000,0.000000,0.700000,0.063500,0.025000,41.907242
50%,160385.500000,5.000000,1.790000,5.710000,1.500000,4.760000,0.581660,0.229000,75.670259
75%,240578.250000,20.000000,7.140000,24.000000,3.600000,10.000000,1.374140,0.541000,94.145336
max,320771.000000,714.290000,550.000000,3520.000000,5380.000000,430.000000,64312.800000,25320.000000,889.380000


Doublons de code ? -> Didier
Valeurs négatives ? -> Salomé
Valeurs > 100 (avec une tolérance) ? -> Aatif
Somme des valeurs des colonnes != 100 (avec une tolérance) ? -> Salomé
Valeurs manquantes ? Aatif + Didier



In [42]:
product_df.head()

,Unnamed: 0,code,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre
0,0,3087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00000
1,1,4530,28.57,28.57,14.29,3.6,3.57,0.00000,0.000,21.40000
2,2,4559,17.86,0.00,17.86,7.1,17.86,0.63500,0.250,38.43500
3,3,16087,57.14,5.36,3.57,7.1,17.86,1.22428,0.482,7.26372
4,4,16094,1.43,NaN,NaN,5.7,8.57,NaN,NaN,84.30000


## Codes dupliqués

In [38]:
columns_to_check = [
    "fat_100g",
    "saturated-fat_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
    "sodium_100g",
    "autre",
]

In [40]:
sum((product_df[["code"] + columns_to_check].groupby("code").nunique() > 1).any(axis=1))

59

Bilan :
- 23 codes manquants,
- 320580 valeurs de code uniques sur 320749 articles avec un code non manquant,
- 59 codes associés à au moins 2 articles avec des compositions différentes.

## Valeurs négatives

In [41]:
mask_neg = (product_df[columns_to_check] < 0).any(axis=1)
product_df[mask_neg]

,Unnamed: 0,code,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre
8582,8582,11213420608,0.00,0.00,-1.20,1.2,2.41,0.38354,0.151000,97.055460
18209,18209,21130493432,0.80,0.00,-0.80,0.8,0.80,0.87376,0.344000,97.182240
23784,23784,28400231053,33.33,13.33,0.00,-6.7,NaN,6.43382,2.533000,51.073180
33781,33781,36800416727,46.43,8.93,3.57,3.6,-3.57,0.99822,0.393000,39.648780
115310,115310,4029816,0.00,NaN,NaN,NaN,-500.00,25.40000,10.000000,564.600000
117739,117739,608866999263,3.57,0.00,-3.57,3.6,7.14,0.95250,0.375000,87.932500
146284,146284,789280259062,13.33,3.33,-6.67,6.7,NaN,2.03200,0.800000,80.478000
150858,150858,813922021028,6.25,1.25,-6.25,1.2,1.25,1.19380,0.470000,94.636200
164030,164030,856336001538,21.43,3.57,-17.86,17.9,17.86,1.93294,0.761000,54.406060
169119,169119,875208001230,0.00,NaN,0.00,NaN,-800.00,7.62000,3.000000,889.380000


In [60]:
sum(mask_neg)

11

## Salt et sodium

In [62]:
ratio_col = product_df["salt_100g"] / product_df["sodium_100g"]

In [67]:
expected_ratio = ratio_col[~ratio_col.isna()].mean()

In [74]:
expected_salt_100g = product_df["sodium_100g"] * expected_ratio

mask_inf = expected_salt_100g < product_df["salt_100g"] - 0.3
mask_sup = expected_salt_100g > product_df["salt_100g"] + 0.3

product_df[mask_inf | mask_sup]

,Unnamed: 0,code,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre
21632,21632,24600017008,0.0,NaN,NaN,NaN,0.0,102.0,40.0,0.0
109154,109154,96619911936,0.0,NaN,NaN,NaN,0.0,107.0,42.0,0.0


## Fat et saturated-fat

In [93]:
fat_mask = product_df["saturated-fat_100g"] > product_df["fat_100g"]
print(sum(fat_mask))

na_fat_mask = product_df["fat_100g"].isna() & (~ product_df["saturated-fat_100g"].isna())
product_df[na_fat_mask]

354


,Unnamed: 0,code,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre


In [92]:
product_df["fat_100g"] = np.where(
    na_fat_mask,
    product_df["saturated-fat_100g"],
    product_df["fat_100g"],
)

In [89]:
product_df.head()

,Unnamed: 0,code,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre
0,0,3087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00000
1,1,4530,28.57,28.57,14.29,3.6,3.57,0.00000,0.000,21.40000
2,2,4559,17.86,0.00,17.86,7.1,17.86,0.63500,0.250,38.43500
3,3,16087,57.14,5.36,3.57,7.1,17.86,1.22428,0.482,7.26372
4,4,16094,1.43,NaN,NaN,5.7,8.57,NaN,NaN,84.30000


## Somme à 100 grammes

In [94]:
columns_to_sum = [
    "fat_100g",
    "sugars_100g",
    "fiber_100g",
    "proteins_100g",
    "salt_100g",
    "autre",
]

In [100]:
product_df["total_weight_100g"] = product_df[columns_to_sum].sum(axis=1, skipna=True)

display(product_df)

,Unnamed: 0,code,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre,total_weight_100g
0,0,3087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00000,100.000
1,1,4530,28.57,28.57,14.29,3.6,3.57,0.00000,0.000,21.40000,71.430
2,2,4559,17.86,0.00,17.86,7.1,17.86,0.63500,0.250,38.43500,99.750
3,3,16087,57.14,5.36,3.57,7.1,17.86,1.22428,0.482,7.26372,94.158
4,4,16094,1.43,NaN,NaN,5.7,8.57,NaN,NaN,84.30000,100.000
...,...,...,...,...,...,...,...,...,...,...,...
320767,320767,9948282780603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00000,100.000
320768,320768,99567453,0.00,0.00,0.00,0.0,0.00,0.00000,0.000,100.00000,100.000
320769,320769,9970229501521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00000,100.000
320770,320770,9980282863788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.00000,100.000


In [102]:
too_high_mask = product_df["total_weight_100g"] > 101
sum(too_high_mask)

888

In [103]:
too_low_mask = product_df["total_weight_100g"] < 99
sum(too_low_mask)

136932

# Avec dask

In [53]:
import dask.dataframe as dd  
product_ddf = dd.read_csv(product_file_path, dtype={'code': 'object'})  


In [54]:
describe_df = product_df.describe()

In [55]:
display(describe_df)

,Unnamed: 0,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre
count,320772.000000,243891.000000,229554.000000,244971.000000,200886.000000,259922.000000,255510.000000,255463.000000,320772.000000
mean,160385.500000,12.730379,5.129932,16.003484,2.862111,7.075940,2.028624,0.798815,65.861511
std,92599.044612,17.578747,8.014238,22.327284,12.867578,8.409054,128.269454,50.504428,32.091021
min,0.000000,0.000000,0.000000,-17.860000,-6.700000,-800.000000,0.000000,0.000000,0.000000
25%,80192.750000,0.000000,0.000000,1.300000,0.000000,0.700000,0.063500,0.025000,41.907242
50%,160385.500000,5.000000,1.790000,5.710000,1.500000,4.760000,0.581660,0.229000,75.670259
75%,240578.250000,20.000000,7.140000,24.000000,3.600000,10.000000,1.374140,0.541000,94.145336
max,320771.000000,714.290000,550.000000,3520.000000,5380.000000,430.000000,64312.800000,25320.000000,889.380000


In [56]:
mask_neg = (product_ddf[columns_to_check] < 0).any(axis=1)
neg_ddf = product_ddf[mask_neg]

In [121]:
pandas_mask_neg = (product_df[columns_to_check] < 0).any(axis=1)
pandas_neg_ddf = product_df[pandas_mask_neg]

In [122]:
print(neg_ddf)

Dask DataFrame Structure:
              Unnamed: 0    code fat_100g saturated-fat_100g sugars_100g fiber_100g proteins_100g salt_100g sodium_100g    autre
npartitions=1                                                                                                                   
                   int64  string  float64            float64     float64    float64       float64   float64     float64  float64
                     ...     ...      ...                ...         ...        ...           ...       ...         ...      ...
Dask Name: getitem, 6 graph layers


In [123]:
pandas_neg_ddf

,Unnamed: 0,code,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre,total_weight_100g
8582,8582,11213420608,0.00,0.00,-1.20,1.2,2.41,0.38354,0.151000,97.055460,99.849000
18209,18209,21130493432,0.80,0.00,-0.80,0.8,0.80,0.87376,0.344000,97.182240,99.656000
23784,23784,28400231053,33.33,13.33,0.00,-6.7,NaN,6.43382,2.533000,51.073180,84.137000
33781,33781,36800416727,46.43,8.93,3.57,3.6,-3.57,0.99822,0.393000,39.648780,90.677000
115310,115310,4029816,0.00,NaN,NaN,NaN,-500.00,25.40000,10.000000,564.600000,90.000000
117739,117739,608866999263,3.57,0.00,-3.57,3.6,7.14,0.95250,0.375000,87.932500,99.625000
146284,146284,789280259062,13.33,3.33,-6.67,6.7,NaN,2.03200,0.800000,80.478000,95.870000
150858,150858,813922021028,6.25,1.25,-6.25,1.2,1.25,1.19380,0.470000,94.636200,98.280000
164030,164030,856336001538,21.43,3.57,-17.86,17.9,17.86,1.93294,0.761000,54.406060,95.669000
169119,169119,875208001230,0.00,NaN,0.00,NaN,-800.00,7.62000,3.000000,889.380000,97.000000


In [124]:
neg_ddf.compute()

,Unnamed: 0,code,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,autre
8582,8582,11213420608,0.00,0.00,-1.20,1.2,2.41,0.38354,0.151000,97.055460
18209,18209,21130493432,0.80,0.00,-0.80,0.8,0.80,0.87376,0.344000,97.182240
23784,23784,28400231053,33.33,13.33,0.00,-6.7,NaN,6.43382,2.533000,51.073180
33781,33781,36800416727,46.43,8.93,3.57,3.6,-3.57,0.99822,0.393000,39.648780
115310,115310,4029816,0.00,NaN,NaN,NaN,-500.00,25.40000,10.000000,564.600000
117739,117739,608866999263,3.57,0.00,-3.57,3.6,7.14,0.95250,0.375000,87.932500
146284,146284,789280259062,13.33,3.33,-6.67,6.7,NaN,2.03200,0.800000,80.478000
150858,150858,813922021028,6.25,1.25,-6.25,1.2,1.25,1.19380,0.470000,94.636200
164030,164030,856336001538,21.43,3.57,-17.86,17.9,17.86,1.93294,0.761000,54.406060
169119,169119,875208001230,0.00,NaN,0.00,NaN,-800.00,7.62000,3.000000,889.380000


In [59]:
len(neg_ddf)

11